In [1]:
import torch
from torch import nn

from luadseg.models.foundation_models import load_model
from luadseg.utils import get_device

Try forward intermediate method, see the timm doc

In [2]:
device = get_device(gpu_id=0)

Using cuda:0


In [3]:
model, preprocess, embedding_dim, autocast_dtype = load_model("UNI2", device=device, apply_torch_scripting=False)

In [4]:
type(model)

timm.models.vision_transformer.VisionTransformer

In [5]:
len(model.blocks)

24

In [6]:
x = torch.randn(1, 3, 224, 224).to(device)

In [7]:
feat = model.forward_intermediates(x, indices=[5, 20, 23], norm=True, stop_early=False, intermediates_only=True)

In [8]:
len(feat)

3

In [9]:
feat[2].shape

torch.Size([1, 1536, 16, 16])

In [ ]:
model.named_parameters()

In [ ]:
feat[0].shape, feat[1][1].shape

In [ ]:
model_without_last_block = torch.nn.Sequential(*model.blocks[:-1])

In [ ]:
model.cls_token

In [ ]:
# Dummy input image
x = torch.randn(1, 3, 224, 224).to(device)  # B x 3 x H x W

# Forward through original model
model.eval()
with torch.no_grad():
    y = model.forward_features(x)  # Shape: [B, L, C]
    y_cls = model(x)

# Forward through wrapper
with torch.no_grad():
    feat = model.forward_intermediates(x, indices=[5, 23], norm=True, stop_early=False)
    y_skips = feat[0]          # Last layer [B, L, C]
    y_cls_skips = feat[0][:, 0, :]  # CLS token [B, C]

# Compare the final outputs
diff = torch.abs(y - y_skips).max()
print(f"Max absolute difference: {diff.item()}")

# Optional: check if close enough
assert torch.allclose(y, y_skips, atol=1e-5), "Mismatch between model and wrapper"
print("✅ Final transformer output matches.")

# Compare the final outputs
diff = torch.abs(y_cls - y_cls_skips).max()
print(f"Max absolute difference: {diff.item()}")


In [ ]:
y_skips.shape, y.shape  # Both should be [B, L, C] where L is the number of tokens and C is the embedding dimension

In [ ]:
y_skips

In [ ]:
y